# IST736 HW1
### Name: Lu Guo


## 1. Data exploration

In [1]:
import pandas as pd
## Import data from google drive
# from google.colab import drive
# drive.mount('/content/drive')
# file_path1 = '/content/drive/My Drive/Colab Notebooks/IST 736/HW1/people-dict.csv'
# file_path2 = '/content/drive/My Drive/Colab Notebooks/IST 736/HW1/happydb_data_cleaned_hm.csv'
# people = pd.read_csv(file_path1, header=None)
# hm = pd.read_csv(file_path2)


## Or import data from computer
people = pd.read_csv('/Users/guolu/Documents/Course/IST 736/HW1/people-dict.csv', header=None)
hm = pd.read_csv("/Users/guolu/Documents/Course/IST 736/HW1/raw.githubusercontent.com_megagonlabs_HappyDB_master_happydb_data_cleaned_hm.csv")

In [2]:
hm.head(2)

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection


In [ ]:
print(len(hm))
print(len(people))

100535
239


In [ ]:
# convert people_dic to dictionary, the values are 0
people_dic = people.set_index(0).T.to_dict('list')
for key in people_dic:
    people_dic[key] = 0

## 2. Data cleaning

In [ ]:
# remove duplicate rows in happy moment
print(len(hm))
hm = hm.drop_duplicates(subset=['original_hm'])
# reset index
hm = hm.reset_index(drop=True)
print(len(hm))

100535
96557


In [ ]:
# convert all the clean text to lower case, so that we can match the words in dictionary
hm['cleaned_hm'] = hm['cleaned_hm'].str.lower()

## 3. Question (1)

Use this lexicon to find the top three social relationships mentioned in happy moments, e.g. spouse, parents, children, friends, or someone else.

In [ ]:
# If a sentence mention a same relationship, count it as multiple times.

import re
for happy_moment in hm['cleaned_hm']:
    for key in people_dic:
      relations = re.findall(r'\b' + re.escape(key) + r'\b', happy_moment) #since there are some regular expression in the people_dic, so I use re.findall to find the match
      people_dic[key] += len(relations)

In [ ]:
# return the top 3 people who are mentioned most frequently
people_dic = sorted(people_dic.items(), key=lambda x: x[1], reverse=True)
people_dic[:3]

[('friend', 5847), ('friends', 4577), ('family', 4385)]

In [ ]:
# If a sentence mention a same relationship, count it as only once.
people_dic1 = people.set_index(0).T.to_dict('list')
for key in people_dic1:
    people_dic1[key] = 0

for happy_moment in hm['cleaned_hm']:
    for key in people_dic1:
        if len(re.findall(r'\b' + re.escape(key) + r'\b', happy_moment)) > 0: #since there are some regular expression in the people_dic, so I use re.findall to find the match
            people_dic1[key] += 1

In [ ]:
people_dic1 = sorted(people_dic1.items(), key=lambda x: x[1], reverse=True)
people_dic1[:3]

[('friend', 5605), ('friends', 4276), ('family', 3958)]

In [ ]:
# compare the two dictionaries, return the keys that have different values
# for key in people_dic:
#     if key in people_dic1 and people_dic[key] != people_dic1[key]:
#         print(key)

#     if key not in people_dic1:
#       print(key)

Answer: the top three social relationships mentioned in happy moments are friend, friends, and family.

## 4. Question (2)

To answer the question “with whom do people spend happy moments?”, the “people dictionary” has one strength aspect and two weekness aspects.

The strength is that the dictionary is very comprehensive. It contains almost all the common social relationships, including some abbreviation, e.g. "bf".

The first weekeness aspect is that the dictionary contains some words are not social relationships, e.g. "men", "guy". The second weekeness aspect is that the dictionary divide the singular and plural forms of the same relationship into different words, e.g. "friend" and "friends".


## 5. Questions (3)

I use a dictionary and set some new categories to merge the words that describe the same relationship. For example:
 'aunt': 'family',
 'auntie': 'family',
 'aunties': 'family'.

 For the words that are not social relationships, I use "other" category to describe them.

In [ ]:
updated_people_dict = {
 'aunt': 'family',
 'auntie': 'family',
 'aunties': 'family',
 'aunts': 'family',
 'aunty': 'family',
 'babies': 'other',
 'baby': 'other',
 'bae': 'other',
 'best friend': 'friend',
 'bestfriend': 'friend',
 'bestfriends': 'friend',
 'bestie': 'friend',
 'besties': 'friend',
 'bf': 'lover',
 'bff': 'friend',
 'bffs': 'friend',
 'boy': 'stranger',
 'boyfriend': 'lover',
 'boys': 'stranger',
 'bro': 'friend',
 'brother': 'family',
 'brother-in-law': 'family',
 'brothers': 'family',
 'buddies': 'friend',
 'buddy': 'friend',
 'child': 'family',
 'children': 'family',
 'close friend': 'friend',
 'co-worker': 'colleague',
 'colleague': 'colleague',
 'cousin': 'family',
 'cousins': 'family',
 'coworker': 'colleague',
 'dad': 'family',
 'daddy': 'family',
 'dads': 'family',
 'daughter': 'family',
 'daughter-in-law': 'family',
 'daughters': 'family',
 'dudes': 'friend',
 'elder brother': 'family',
 'elder son': 'family',
 'eldest': 'other',
 'eldest son': 'family',
 'everybody': 'stranger',
 'everyone': 'stranger',
 'ex-boyfriend': 'past relationship',
 'ex-girlfriend': 'past relationship',
 'ex-husband': 'past relationship',
 'ex-wife': 'past relationship',
 'families': 'family',
 'family': 'family',
 'father': 'family',
 'father-in-law': 'family',
 'fiance': 'lover',
 'fiancee': 'lover',
 'fiancé': 'lover',
 'fiancée': 'lover',
 'folks': 'stranger',
 'friend': 'friend',
 'friends': 'friend',
 'gf': 'lover',
 'girl': 'stranger',
 'girlfriend': 'lover',
 'girls': 'stranger',
 'good friend': 'friend',
 'grand-daughter': 'family',
 'grandaughter': 'family',
 'grandchild': 'family',
 'grandchildren': 'family',
 'granddaughter': 'family',
 'granddaughters': 'family',
 'grandfather': 'family',
 'grandfathers': 'family',
 'grandma': 'family',
 'grandmas': 'family',
 'grandmother': 'family',
 'grandmothers': 'family',
 'grandpa': 'family',
 'grandparents': 'family',
 'grandson': 'family',
 'grandsons': 'family',
 'grannies': 'family',
 'granny': 'family',
 'great-grandchildren': 'family',
 'great-granddaughter': 'family',
 'great-grandfather': 'family',
 'great-grandmother': 'family',
 'great-grandson': 'family',
 'great-great-grandson': 'family',
 'guy': 'stranger',
 'guys': 'stranger',
 'hubby': 'family',
 'husband': 'family',
 'in-law': 'family',
 'in-laws': 'family',
 'infant': 'family',
 'infants': 'family',
 'inlaws': 'family',
 'kid': 'family',
 'kids': 'family',
 'lover': 'lover',
 'mama': 'family',
 'mamma': 'family',
 'man': 'stranger',
 'mom': 'family',
 'momma': 'family',
 'mommy': 'family',
 'moms': 'family',
 'mother': 'family',
 'mother-in-law': 'family',
 'mothers': 'family',
 'mum': 'family',
 'neice': 'family',
 'neighbor': 'neighbor',
 'neighbors': 'neighbor',
 'neighbour': 'neighbor',
 'neighbours': 'neighbor',
 'nephew': 'family',
 'nephews': 'family',
 'newborn': 'family',
 'newborns': 'family',
 'niece': 'family',
 'nieces': 'family',
 'older brother': 'family',
 'oldest son': 'family',
 'only son': 'family',
 'pals': 'friend',
 'papa': 'family',
 'pappa': 'family',
 'parent': 'family',
 'parents': 'family',
 'partner': 'colleague',
 'partners': 'colleague',
 'people': 'stranger',
 'person': 'stranger',
 'ppl': 'other',
 'preschoolers': 'other',
 'second son': 'family',
 'sister': 'family',
 'sister-in-law': 'family',
 'sisters': 'family',
 'sisters-in-law': 'family',
 'somebody': 'stranger',
 'someone': 'stranger',
 'son': 'family',
 'son-in-law': 'family',
 'sons': 'family',
 'step-father': 'family',
 'step-mother': 'family',
 'stepdaughter': 'family',
 'stepfather': 'family',
 'stepmother': 'family',
 'stepson': 'family',
 'them': 'other',
 'toddler': 'other',
 'toddlers': 'other',
 'uncle': 'family',
 'uncles': 'family',
 'wife': 'family',
 'woman': 'other',
 'younger brother': 'family',
 'younger son': 'family',
 'youngest son': 'family',
 'fathers': 'family',
 'siblings': 'family',
 'chick': 'other',
 'chicks': 'other',
 'sibling': 'family',
 'babe': 'other',
 'girlfriends': 'lover',
 'brunette': 'other',
 'babes': 'other',
 'boyfriends': 'lover',
 'blonde': 'other',
 'bby': 'family',
 'cutie': 'other',
 'blond': 'other',
 'hotties': 'other',
 'blondes': 'other',
 'brunettes': 'other',
 'blonds': 'other',
 'women': 'other',
 'men': 'other',
 'peeps': 'friend',
 'great-grandparents': 'family',
 'stepdad': 'family',
 'coworkers': 'colleague',
 'co-workers': 'colleague',
 'grandkids': 'family',
 'every1': 'other',
 'some1': 'stranger',
 'tenns': 'other',
 'teens': 'other',
 'teen': 'other',
 'teenagers': 'other',
 'teenager': 'other',
 'my crush': 'lover',
 'my ex': 'past relationship',
 'his crush': 'lover',
 'her crush': 'lover',
 'his ex': 'past relationship',
 'her ex': 'past relationship',
 'only child': 'family',
 'twin brother': 'family',
 'ladies': 'stranger',
 'redheads': 'other',
 'only daughter': 'family',
 'mommies': 'family',
 'adults': 'other',
 'eldest daughter': 'family',
 'youngest child': 'family',
 'youngest daughter': 'family',
 'eldest child': 'family',
 'sis': 'family',
 'stepmom': 'family',
 'mummy': 'family',
 'kiddos': 'family',
 'fellas': 'friend',
 'fella': 'friend',
 'bloke': 'other',
 'lad': 'friend',
 'colleagues': 'colleague',
 'blokes': 'other',
 'lads': 'friend',
 'adult': 'other',
 'childhood friend': 'past relationship',
 'customer': 'business',
 'client': 'business',
 'stranger': 'stranger',
 'strangers': 'stranger',
 'co worker': 'colleague',
 'love of my life': 'lover',
 '[\\w]*mate(s?)': 'lover',
 'act(or|ress)': 'stranger',
 'lady': 'stranger',
 'teacher': 'teacher',
 'celebrity': 'stranger',
 'professor': 'teacher'
    }

In [ ]:
import re
result = {"family":0, "lover":0, "friend":0, "colleague": 0, "past relationship":0, "stranger": 0, "neighbor":0, "teacher":0, "business":0,"other":0}
for happy_moment in hm['cleaned_hm']:
    for key,value in updated_people_dict.items():
      relations = re.findall(r'\b' + re.escape(key) + r'\b', happy_moment) #since there are some regular expression in the people_dic, so I use re.findall to find the match
      result[value] += len(relations)

In [ ]:
# return the top 3 people who are mentioned most frequently
result = sorted(result.items(), key=lambda x: x[1], reverse=True)
result

[('family', 33590),
 ('friend', 11606),
 ('stranger', 5718),
 ('other', 4211),
 ('lover', 3804),
 ('colleague', 1321),
 ('neighbor', 501),
 ('business', 374),
 ('teacher', 327),
 ('past relationship', 236)]

The top 3 relationships are family, friend, and stranger. Because in my dictionary, other type is not relationship.

## 5. Question (4)

In [ ]:
# find all family people in the updated_people_dict
family = []
for key,value in updated_people_dict.items():
    if value == "family":
        family.append(key)
family

['aunt',
 'auntie',
 'aunties',
 'aunts',
 'aunty',
 'brother',
 'brother-in-law',
 'brothers',
 'child',
 'children',
 'cousin',
 'cousins',
 'dad',
 'daddy',
 'dads',
 'daughter',
 'daughter-in-law',
 'daughters',
 'elder brother',
 'elder son',
 'eldest son',
 'families',
 'family',
 'father',
 'father-in-law',
 'grand-daughter',
 'grandaughter',
 'grandchild',
 'grandchildren',
 'granddaughter',
 'granddaughters',
 'grandfather',
 'grandfathers',
 'grandma',
 'grandmas',
 'grandmother',
 'grandmothers',
 'grandpa',
 'grandparents',
 'grandson',
 'grandsons',
 'grannies',
 'granny',
 'great-grandchildren',
 'great-granddaughter',
 'great-grandfather',
 'great-grandmother',
 'great-grandson',
 'great-great-grandson',
 'hubby',
 'husband',
 'in-law',
 'in-laws',
 'infant',
 'infants',
 'inlaws',
 'kid',
 'kids',
 'mama',
 'mamma',
 'mom',
 'momma',
 'mommy',
 'moms',
 'mother',
 'mother-in-law',
 'mothers',
 'mum',
 'neice',
 'nephew',
 'nephews',
 'newborn',
 'newborns',
 'niece',
 '

Filter stopwords and then use POS tagging to filter the content words.

non-content words are: 'CD', 'WDT', 'WRB', 'PRP', 'WP', 'UH', 'DT', 'WP\$', 'MD', 'IN', 'PRP\$', 'TO', 'CC', 'FW', 'RP'.

    
content words are: 'RB', 'NNS', 'VB', 'RBR', 'VBN', 'JJS', 'JJ', 'JJR', 'VBG', 'VBD', 'VBZ', 'NN', 'JJR', 'VBP', 'RBS', 'NNP'.

In [ ]:
# tokenize the cleaned_hm, remove the stop words and non-content words
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
# add punctuations to stop words
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','%'])

nltk.download('averaged_perceptron_tagger')
content_works_tag = ['RB','NNS','VB','RBR','VBN','JJS','JJ','JJR','VBG','VBD','VBZ','NN','JJR','VBP','RBS','NNP']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
result_content_words = []
for i in range(len(hm)):
    tokens = nltk.word_tokenize(hm['cleaned_hm'][i])
    tokens = [w for w in tokens if w not in stop_words] # remove stop words
    tagged_tokens = nltk.pos_tag(tokens)
    content_words_tokens = [word for word, pos in tagged_tokens if pos in content_works_tag] # remove non-content words
    # extract five context words before and after the word in family list
    cur_content_end = 0
    for word in content_words_tokens:
        if word in family:
            index = content_words_tokens.index(word)
            cur_content_start = max(0, index-5, cur_content_end+1) # if the index is less than 5, then start from 0; if the tokens has been extracted, then start from the last index
            cur_content_end = min(len(content_words_tokens), index+6)
            result_content_words.extend(content_words_tokens[cur_content_start:index] + content_words_tokens[index+1:cur_content_end])

In [ ]:
result_content_words

['got',
 'marks',
 'examination',
 'butterfly',
 'display',
 'crohn',
 'conservatory',
 'gift',
 'elder',
 'really',
 'surprising',
 'birthday',
 'enjoyed',
 'cupcake',
 'wars',
 'teen',
 'performed',
 'well',
 'test',
 'preparation',
 'called',
 'abroad',
 'morning',
 'long',
 'years',
 'occassion',
 'members',
 'start',
 'walk',
 'bed',
 'long',
 'time',
 'bed',
 'afternoon',
 'sweet',
 'cuddle',
 'close',
 'woke',
 'fantastic',
 'breakfast',
 'eggs',
 'special',
 'time',
 'gave',
 'suprise',
 'visit',
 'house',
 'yesterday',
 'movies',
 'today',
 'first',
 'time',
 'long',
 'time',
 'airport',
 'fun',
 'good',
 'conversation',
 'way',
 'waved',
 'said',
 'mama',
 'first',
 'time',
 'first',
 'time',
 'came',
 'home',
 'morning',
 'grew',
 'together',
 'went',
 'school',
 "'re",
 "'ve",
 'playing',
 'game',
 'together',
 'almost',
 'keeps',
 'patting',
 'younger',
 'daughter',
 'head',
 'keeps',
 'patting',
 'younger',
 'daughter',
 'head',
 'lonely',
 'long',
 'day',
 'work',
 'call

In [ ]:
from collections import Counter

# Count word frequencies
word_frequencies = Counter(result_content_words)

# Get the 100 most common words
counter = 0
for word, frequency in word_frequencies.most_common(100):
    print(word+",", end = " ")
    counter += 1
    if counter % 10 == 0:  # print ten words a line
        print("")


happy, time, made, day, got, birthday, home, new, went, first, 
dinner, came, last, see, school, old, house, family, night, good, 
work, n't, visit, get, go, really, friends, nice, today, gave, 
together, great, told, able, year, long, month, mother, yesterday, took, 
years, baby, morning, moment, week, spend, trip, gift, going, members, 
months, party, weekend, phone, lunch, bought, said, favorite, play, back, 
felt, enjoyed, daughter, son, little, finally, lot, job, well, friend, 
sister, much, brought, fun, game, vacation, called, started, ago, surprise, 
take, car, life, park, love, movie, kids, big, wedding, found, 
days, surprised, spent, feel, saw, born, getting, seeing, easter, wife, 


From the first 100 content words, we can see which activities lead to happy moments, and when and where happy moments typically occur.

Activities: The activities lead to happy moments: celebrate "birthday", have "lunch" or "dinner", "work", during "trip" and "vacation", receive "gift", participate "party" or "wedding", play "game", watch "movie".

Location: The happy moments happened in "school", "house", "car", and "park".

Time: The happy moments happened when "night", "morning", and "weekend".

## 6. Claim
I used ChatGPT to help me fix codes when error occurs. I also used ChatGPT to help me classify some words into different relationship categories when I don't understand the words, e.g. "ppl".